Install and Import Dependencies

In [ ]:
!pip install torch==1.8.2 torchvision==0.9.2 torchaudio===0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cpu

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!cd yolov5 & pip install -r requirements.txt 

In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

Load Model

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\M.Amin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-19 Python-3.9.6 torch-1.8.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Real Time Detections with yolo

In [26]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Make our own training set

In [8]:
import uuid   # Unique identifier
import os
import time

Before doing the below, you should make directory named data, and in it, create two directories: images and labels

In [28]:
IMAGES_PATH = os.path.join('data', 'images') #/data/images 
labels = ['awake', 'drowsy']
number_imgs = 20 #It is the number of drowsy and awake sample for better model we should increase this

With the below script, we can collect our data

In [29]:
cap = cv2.VideoCapture(0)
# Loop through labels
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    
    # Loop through image range
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        
        # Webcam feed
        ret, frame = cap.read()
        
        # Naming out image path
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        
        # Writes out image to file 
        cv2.imwrite(imgname, frame)
        
        # Render to the screen
        cv2.imshow('Image Collection', frame)
        
        # 2 second delay between captures
        time.sleep(2)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

To give labels to these collected pictures, we use software downloaded from the below link:

In [ ]:
!git clone https://github.com/tzutalin/labelImg

Cloning into 'labelImg'...
error: RPC failed; curl 56 OpenSSL SSL_read: Connection was reset, errno 10054
error: 1198 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [32]:
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

  Using cached PyQt5-5.15.7-cp37-abi3-win_amd64.whl (6.8 MB)
  Using cached lxml-4.9.1-cp39-cp39-win_amd64.whl (3.6 MB)
     -------------------------------------- 50.1/50.1 MB 403.3 kB/s eta 0:00:00
     -------------------------------------- 78.3/78.3 kB 167.7 kB/s eta 0:00:00


Train our model

Before doing the script below, we should make a dataset.yml file with classes that are in the data\labels directory
Increase the epochs to get a better model, but it will be time-consuming

In [32]:
!cd yolov5 && python train.py --img 320 --batch 16 --epochs 200 --data dataset.yml --weights yolov5s.pt --workers 2

train: weights=yolov5s.pt, cfg=, data=dataset.yml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 
YOLOv5  v6.2-21-g1cd3e75 Python-3.9.6 torch-1.8.2+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, sc

Load Custom Model

In [33]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp7/weights/last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\M.Amin/.cache\torch\hub\master.zip
YOLOv5  2022-8-19 Python-3.9.6 torch-1.8.2+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [34]:
img = os.path.join('data', 'images', 'awake.c810e0d7-1f87-11ed-bf96-40e230cb1c34.jpg')

In [35]:
results = model(img)

In [36]:
results.print()

image 1/1: 480x640 1 awake
Speed: 9.0ms pre-process, 303.0ms inference, 2.0ms NMS per image at shape (1, 3, 480, 640)


In [38]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()